#Pandas Patch In Action 

## import packages and download test data 

In [27]:
from pandas_patch import *

In [30]:
%psource structure

In [11]:
def get_test_df_complete():
    """ get the full test dataset from Lending Club open source database,
    the purpose of this fuction is to be used in a demo ipython notebook """
    import requests
    from zipfile import ZipFile
    from StringIO import StringIO
    zip_to_download = "https://resources.lendingclub.com/LoanStats3b.csv.zip"
    r = requests.get(zip_to_download)
    zipfile = ZipFile(StringIO(r.content))
    file_csv = zipfile.namelist()[0]
    df = pd.read_csv(zipfile.open(file_csv), skiprows =[0], na_values = ['n/a','N/A',''],
     parse_dates = ['issue_d','last_pymnt_d','next_pymnt_d','last_credit_pull_d'] )
    zipfile.close()
    df = df[:-2]
    nb_row = float(len(df.index))
    df['na_col'] = np.nan
    df['constant_col'] = 'constant'
    df['duplicated_column'] = df.id
    df['many_missing_70'] = np.nan
    df.loc[1:int(0.3*nb_row),'many_missing_70'] = 1
    df['bad'] = 1
    index_good = df['loan_status'].isin(['Fully Paid', 'Current','In Grace Period'])
    df.loc[index_good,'bad'] = 0
    return df 


In [12]:
# ipython tips
# with psource you can see the source code of a function
%psource pandas_patch

?nacount # to get info about the functions and docs

In [13]:
#df = get_test_df_complete()

# because no wifi connection 
df = get_test_df_complete()

/Users/efourrier/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1164: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [19]:
df.columns

Index([u'id', u'member_id', u'loan_amnt', u'funded_amnt', u'funded_amnt_inv', u'term', u'int_rate', u'installment', u'grade', u'sub_grade', u'emp_title', u'emp_length', u'home_ownership', u'annual_inc', u'is_inc_v', u'issue_d', u'loan_status', u'pymnt_plan', u'url', u'desc', u'purpose', u'title', u'zip_code', u'addr_state', u'dti', u'delinq_2yrs', u'earliest_cr_line', u'inq_last_6mths', u'mths_since_last_delinq', u'mths_since_last_record', u'open_acc', u'pub_rec', u'revol_bal', u'revol_util', u'total_acc', u'initial_list_status', u'out_prncp', u'out_prncp_inv', u'total_pymnt', u'total_pymnt_inv', u'total_rec_prncp', u'total_rec_int', u'total_rec_late_fee', u'recoveries', u'collection_recovery_fee', u'last_pymnt_d', u'last_pymnt_amnt', u'next_pymnt_d', u'last_credit_pull_d', u'collections_12_mths_ex_med', u'mths_since_last_major_derog', u'policy_code', u'na_col', u'constant_col', u'duplicated_column', u'many_missing_70', u'bad'], dtype='object')

## Basic data cleaning and exploration 

### Basic helpers 

In [20]:
df.nrow()

188123

In [21]:
df.ncol()

57

In [22]:
df.dfnum() #identify numeric variables

['member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'installment',
 'annual_inc',
 'dti',
 'delinq_2yrs',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'total_acc',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_amnt',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'na_col',
 'many_missing_70',
 'bad']

In [23]:
df.dfchar() # identify character variables 

['id',
 'term',
 'int_rate',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'is_inc_v',
 'loan_status',
 'pymnt_plan',
 'url',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'earliest_cr_line',
 'revol_util',
 'initial_list_status',
 'constant_col',
 'duplicated_column']

In [24]:
timeit df.factors()

1 loops, best of 3: 1.98 s per loop


In [118]:
df.nacount(axis = 0) # counting the number of missing values per column 

,Nanumber,Napercentage
id,0,0.000000
member_id,0,0.000000
loan_amnt,0,0.000000
funded_amnt,0,0.000000
funded_amnt_inv,0,0.000000
term,0,0.000000
int_rate,0,0.000000
installment,0,0.000000
grade,0,0.000000
sub_grade,0,0.000000


In [119]:
df.nacount(axis = 1) # count the number of missing values per rows 

,Nanumber,Napercentage
0,4,0.071429
1,4,0.071429
2,2,0.035714
3,1,0.017857
4,2,0.035714
5,3,0.053571
6,2,0.035714
7,3,0.053571
8,4,0.071429
9,4,0.071429


In [96]:
df.constantcol() # find the constant columns 

['policy_code']

In [97]:
df.findupcol() # find the duplicate columns 

[]

In [138]:
timeit df.detectkey(pct = )

1 loops, best of 3: 1.37 s per loop


In [37]:
timeit df.apply(lambda x: len(pd.unique(x)))

1 loops, best of 3: 7.66 s per loop


In [38]:
timeit df.count_unique()

1 loops, best of 3: 7.77 s per loop


In [20]:
df.manymissing(a = 0.7)

Index([u'desc', u'mths_since_last_record', u'mths_since_last_major_derog'], dtype='object')

### Summary of strucuture, data info and cleaning functions

In [31]:
timeit df.structure()

1 loops, best of 3: 11.1 s per loop


In [22]:
df.psummary(dynamic = True)

there are 0 duplicated rows

the columns with more than 70.00% manymissing values:
['desc', 'mths_since_last_record', 'mths_since_last_major_derog'] 

the columns with less than 5.00% manymissing values are :
['title', 'revol_util', 'last_pymnt_d', 'last_credit_pull_d'] 
 you should fill them with median or most common value 

the detected keys of the dataset are:
['id', 'member_id', 'url'] 

the duplicated columns of the dataset are:
[]

the constant columns of the dataset are:
['policy_code']

the columns with nearzerovariance are:
['pymnt_plan', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'next_pymnt_d', 'last_credit_pull_d', 'collections_12_mths_ex_med']

0.999999954602
0.9999995703
0.999997894256
0.999997750074
0.999995699105
0.990938422868
0.967614558757
0.951902875674
0.875725744027
the columns highly correlated to others to remove are:
['fico_range_low', 'funded_amnt', 'total_pymnt_inv', 'out_prncp_inv', 'funded_amnt_inv', 'id', 'total_pymnt', 'loan_amnt']

t

In [27]:
df.str

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code
0,29003494,31526675,28000,28000,28000,36 months,0.071,866.10,A,A3,...,0,14-Nov,866.10,14-Dec,14-Nov,719,715,0,NaN,1
1,28913233,31436395,3000,3000,3000,36 months,0.117,99.18,B,B4,...,0,14-Oct,99.18,14-Dec,14-Nov,689,685,0,NaN,1
2,27592013,30095159,18000,18000,18000,36 months,0.140,615.03,C,C3,...,0,14-Nov,615.03,14-Dec,14-Nov,699,695,0,27,1
3,28172444,30685592,15000,15000,15000,60 months,0.202,399.08,E,E3,...,0,14-Nov,399.08,14-Dec,14-Nov,664,660,0,70,1
4,20349337,22622030,23275,23275,23275,60 months,0.176,585.61,D,D4,...,0,14-Nov,585.61,15-Jan,14-Nov,709,705,0,57,1
5,27502482,30005666,21000,21000,21000,60 months,0.140,488.42,C,C3,...,0,14-Nov,488.42,14-Dec,14-Nov,744,740,0,NaN,1
6,27702335,30215519,7800,7800,7800,36 months,0.110,255.33,B,B3,...,0,14-Nov,255.33,14-Dec,14-Nov,674,670,0,43,1
7,28032412,30545550,13625,13625,13625,36 months,0.163,480.97,D,D2,...,0,14-Nov,480.97,14-Dec,14-Nov,719,715,0,NaN,1
8,28032523,30545669,17225,17225,17225,60 months,0.140,400.62,C,C3,...,0,14-Nov,400.62,14-Dec,14-Nov,759,755,0,NaN,1
9,28042338,30555469,8150,8150,8150,36 months,0.092,259.82,B,B1,...,0,14-Nov,259.82,14-Dec,14-Nov,769,765,0,NaN,1


In [48]:
%psource structure

In [49]:
df.nacount(axis = 0).Napercentage

id                             0.000000
member_id                      0.000000
loan_amnt                      0.000000
funded_amnt                    0.000000
funded_amnt_inv                0.000000
term                           0.000000
int_rate                       0.000000
installment                    0.000000
grade                          0.000000
sub_grade                      0.000000
emp_title                      0.059915
emp_length                     0.000000
home_ownership                 0.000000
annual_inc                     0.000000
is_inc_v                       0.000000
issue_d                        0.000000
loan_status                    0.000000
pymnt_plan                     0.000000
url                            0.000000
desc                           0.784068
purpose                        0.000000
title                          0.000017
zip_code                       0.000000
addr_state                     0.000000
dti                            0.000000


In [32]:
%timeit df.nacount()

1 loops, best of 3: 239 ms per loop


In [34]:
timeit df.count_unique()

1 loops, best of 3: 7.47 s per loop


In [67]:
df.count()

id                             295987
member_id                      295987
loan_amnt                      295987
funded_amnt                    295987
funded_amnt_inv                295987
term                           295987
int_rate                       295987
installment                    295987
grade                          295987
sub_grade                      295987
emp_title                      278253
emp_length                     295987
home_ownership                 295987
annual_inc                     295987
is_inc_v                       295987
issue_d                        295987
loan_status                    295987
pymnt_plan                     295987
url                            295987
desc                            63913
purpose                        295987
title                          295982
zip_code                       295987
addr_state                     295987
dti                            295987
delinq_2yrs                    295987
earliest_cr_

In [68]:
df.nacount()

,Nanumber,Napercentage
id,0,0.000000
member_id,0,0.000000
loan_amnt,0,0.000000
funded_amnt,0,0.000000
funded_amnt_inv,0,0.000000
term,0,0.000000
int_rate,0,0.000000
installment,0,0.000000
grade,0,0.000000
sub_grade,0,0.000000


In [79]:
1 >= 2

False

In [80]:
df.int_rate.dtype

dtype('float64')

In [127]:
df.sample_df(pct = 0.10).nrow()

29598

In [10]:
df.factors()

1 loops, best of 3: 804 ms per loop


In [15]:
timeit df.detectkey(pct = 0.05)

1 loops, best of 3: 642 ms per loop


In [16]:
timeit df.detectkey2()

1 loops, best of 3: 3.37 s per loop


In [18]:
df.nearzerovar()

                               freq_ratio    nzv  percent_unique zero_var
id                               1.000000  False      100.000000    False
member_id                        1.000000  False      100.000000    False
loan_amnt                        1.319625  False        0.446303    False
funded_amnt                      1.319625  False        0.446303    False
funded_amnt_inv                  1.350674  False        0.457115    False
term                             2.556126  False        0.000676    False
int_rate                         1.049433  False        0.033785    False
installment                      1.211738  False       13.732698    False
grade                            1.042449  False        0.002365    False
sub_grade                        1.019850  False        0.011825    False
emp_title                        1.253420  False       44.487765    False
emp_length                       3.964490  False        0.004054    False
home_ownership                   1.358

Index([u'pymnt_plan', u'total_rec_late_fee', u'recoveries', u'collection_recovery_fee', u'next_pymnt_d', u'last_credit_pull_d', u'collections_12_mths_ex_med'], dtype='object')

In [39]:
def pandas_to_ndarray_wrap(X, copy=True):
    """
    Converts X to a ndarray and provides a function to help convert back
    to pandas object.
    Parameters
    ----------
    X : Series/DataFrame/ndarray
    copy : Boolean
        If True, return a copy.
    Returns
    -------
    Xvals : ndarray
        If X is a Series/DataFrame, then Xvals = X.values,
        if ndarray, Xvals = X
    F : Function
        F(Xvals) = X
    """
    if copy:
        X = X.copy()

    if isinstance(X, pd.Series):
        return X.values, lambda Z: pd.Series(np.squeeze(Z), index=X.index)
    elif isinstance(X, pd.DataFrame):
        return X.values, lambda Z: pd.DataFrame(
            Z, index=X.index, columns=X.columns)
    elif isinstance(X, np.ndarray) or isspmatrix(X):
        return X, lambda Z: Z
    else:
        raise ValueError("Unhandled type: %s" % type(X))

In [40]:
pandas_to_ndarray_wrap(df)

(array([[10159548, 12011167.0, 15000.0, ..., 10159548, nan, 0],
        [10149488, 12001033.0, 4800.0, ..., 10149488, 1.0, 0],
        [10129403, 11981032.0, 7550.0, ..., 10129403, 1.0, 0],
        ..., 
        ['1059224', 1290827.0, 35000.0, ..., '1059224', nan, 0],
        ['1058722', 1290521.0, 12000.0, ..., '1058722', nan, 1],
        ['1058291', 1289878.0, 12000.0, ..., '1058291', nan, 0]], dtype=object),
 <function __main__.<lambda>>)

In [41]:
pandas_to_ndarray_wrap(df)[0]

array([[10159548, 12011167.0, 15000.0, ..., 10159548, nan, 0],
       [10149488, 12001033.0, 4800.0, ..., 10149488, 1.0, 0],
       [10129403, 11981032.0, 7550.0, ..., 10129403, 1.0, 0],
       ..., 
       ['1059224', 1290827.0, 35000.0, ..., '1059224', nan, 0],
       ['1058722', 1290521.0, 12000.0, ..., '1058722', nan, 1],
       ['1058291', 1289878.0, 12000.0, ..., '1058291', nan, 0]], dtype=object)

In [44]:
df.size

10723011

In [45]:
timeit df.duplicated()

1 loops, best of 3: 1.08 s per loop
